In [1]:
%matplotlib qt
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import mne
from mne.datasets import sample
from mne.decoding import (SlidingEstimator, GeneralizingEstimator, Scaler,
                          cross_val_multiscore, LinearModel, get_coef,
                          Vectorizer, CSP)

Sublist_exp1 = ['1','2','5','6','8','9','10','11','12','13','14','15','17','18','19','21','22','23','24','25','26','27','28','29',\
    '31','32','33','34','35','37','38','39','40']

PATH = 'D:\Onedrive\毕业论文\实验数据\实验一\EEGs'

In [2]:
mvpa_epo_dict_part1 = {}
for sub in Sublist_exp1:
    os.chdir(PATH)
    os.chdir('Sub_0' + sub)
    testepo = mne.read_epochs('2_2_epochs_epo.fif', preload = True)
    mvpa_epo = testepo['JoeFace']
    mvpa_epo_dict_part1[sub] = mvpa_epo
    print(mvpa_epo.get_data().shape)


Reading 2_2_epochs_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
331 matching events found
No baseline correction applied
0 projection items activated
(27, 64, 701)
Reading 2_2_epochs_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
418 matching events found
No baseline correction applied
0 projection items activated
(35, 64, 701)
Reading 2_2_epochs_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
386 matching events found
No baseline correction applied
0 projection items activated
(33, 64, 701)
Reading 2_2_epochs_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     500.00 ms
        0 CTF compen

In [3]:
mvpa_epo_dict_part2 = {}
for sub in Sublist_exp1:
    os.chdir(PATH)
    os.chdir('Sub_0' + sub)
    testepo = mne.read_epochs('2_2_epochs_epo.fif', preload = True)
    mvpa_epo = testepo['OtherFace_1']
    mvpa_epo_dict_part2[sub] = mvpa_epo
    print(mvpa_epo.get_data().shape)
# epoch 实例 get_data() 后得到的是 3-Dimension 的数组，(epoch，电极，时间点)
# epoch 实例 get_data() 后得到的是 3-Dimension 的数组，(epoch，电极，时间点)

Reading 2_2_epochs_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
331 matching events found
No baseline correction applied
0 projection items activated
(32, 64, 701)
Reading 2_2_epochs_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
418 matching events found
No baseline correction applied
0 projection items activated
(35, 64, 701)
Reading 2_2_epochs_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
386 matching events found
No baseline correction applied
0 projection items activated
(33, 64, 701)
Reading 2_2_epochs_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     500.00 ms
        0 CTF compen

In [5]:
concated_epochs_part1 = mne.concatenate_epochs(list(mvpa_epo_dict_part1.values()))
concated_epochs_part1_modified = mne.epochs.combine_event_ids(concated_epochs_part1, ['JoeFace'],{'JoeFace_part2' : 34})

Not setting metadata
Not setting metadata
1030 matching events found
Applying baseline correction (mode: mean)
0 bad epochs dropped


In [4]:
concated_epochs_part2 = mne.concatenate_epochs(list(mvpa_epo_dict_part2.values()))
concated_epochs_part2_modified = mne.epochs.combine_event_ids(concated_epochs_part2, ['OtherFace_1'],{'OtherFace_part2' : 35})

Not setting metadata
Not setting metadata
1046 matching events found
Applying baseline correction (mode: mean)
0 bad epochs dropped


In [6]:
concated_epochs_mvpa = mne.concatenate_epochs([concated_epochs_part1_modified, concated_epochs_part2_modified])

Not setting metadata
Not setting metadata
2076 matching events found
Applying baseline correction (mode: mean)
0 bad epochs dropped


In [7]:
concated_epochs_mvpa.equalize_event_counts(concated_epochs_mvpa.event_id)

Dropped 16 epochs: 2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070, 2071, 2072, 2073, 2074, 2075


(<Epochs |  2060 events (all good), -0.2 - 0.5 sec, baseline [-0.2, 0] sec, ~705.2 MB, data loaded,
  'JoeFace_part2': 1030
  'OtherFace_part2': 1030>,
 array([2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070,
        2071, 2072, 2073, 2074, 2075], dtype=int64))

In [8]:
concated_epochs_mvpa.event_id

{'JoeFace_part2': 34, 'OtherFace_part2': 35}

In [9]:
######################################################
# Try to do MVPA in mne.decoding (2)
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
import mne
from mne.datasets import sample
import os
import mne.decoding
from sklearn.preprocessing import StandardScaler
from mne.decoding import SlidingEstimator

from sklearn.svm import SVC

In [10]:
# No need to do Vectorizer() as before
X = concated_epochs_mvpa.get_data(picks= 'eeg')
y = concated_epochs_mvpa.events[:,2]
clf = make_pipeline(StandardScaler(), SVC())

scoring = 'accuracy'
time_decoder = SlidingEstimator(clf, scoring = scoring, n_jobs = -1, verbose = True)

#交叉验证
n_splits = 10
scores = mne.decoding.cross_val_multiscore(time_decoder, X, y , cv = n_splits, n_jobs = -1)

mean_scores = np.mean(scores, axis = 0)
mean_cross_all_times = round(np.mean(scores),3)

print(mean_cross_all_times)


fig, ax = plt.subplots()
plt.ylim(0.3,0.75)
ax.axhline(0.5, color = 'k', label = 'chance')
ax.axvline(0, color = 'k', label = 'zero time')
ax.plot(mvpa_epo.times,mean_scores, label = 'score')

[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   3 out of  10 | elapsed:  1.7min remaining:  3.9min
[Parallel(n_jobs=16)]: Done   6 out of  10 | elapsed:  2.8min remaining:  1.9min


0.555


[Parallel(n_jobs=16)]: Done  10 out of  10 | elapsed:  3.1min finished
